# Algorithm Approach

Using ChatGPT we drive this algorithm that allows for the calculation of the Weakness number

In [1]:
import itertools
import numpy as np

def min_rows_to_cover(matrix):
    m, n = len(matrix), len(matrix[0])
    for r in range(1, m+1):
        for rows in itertools.combinations(range(m), r):
            combined_row = np.zeros(n, dtype=int)
            for row in rows:
                combined_row += np.array(matrix[row])
            if all(combined_row >= 1):
                return np.array(list(rows))
    return np.array([])
# Example usage
K2 = np.array([[ 1,  0,  0,  0,  0,  0,  1,  0,  0],# m1
              [ 1,  0,  0,  0,  0,  0,  0,  1,  0], # m2
              [ 1,  0,  0,  0,  0,  0,  0,  0,  1], # m3
              [ 0,  1,  0,  0,  0,  0,  1,  0,  0], # m4
              [ 0,  1,  0,  0,  0,  0,  0,  1,  0], # m5
              [ 0,  1,  0,  0,  0,  0,  0,  0,  1], # m6
              [ 0,  0,  1,  0,  0,  0,  1,  0,  0], # m7
              [ 0,  0,  1,  0,  0,  0,  0,  1,  0], # m8
              [ 0,  0,  1,  0,  0,  0,  0,  0,  1], # m9
              [ 0,  0,  0,  1,  0,  0,  1,  0,  0], # m10
              [ 0,  0,  0,  1,  0,  0,  0,  1,  0], # m11
              [ 0,  0,  0,  1,  0,  0,  0,  0,  1], # m12
              [ 0,  0,  0,  0,  1,  0,  1,  0,  0], # m13
              [ 0,  0,  0,  0,  1,  0,  0,  1,  0], # m14
              [ 0,  0,  0,  0,  1,  0,  0,  0,  1], # m15
              [ 0,  0,  0,  0,  0,  1,  1,  0,  0], # m16
              [ 0,  0,  0,  0,  0,  1,  0,  1,  0], # m17
              [ 0,  0,  0,  0,  0,  1,  0,  0,  1]])# m18
K2 = np.zeros((18, 9), dtype=int)
result = min_rows_to_cover(K2)
print("The indices of the minimum number of rows needed:", result+1)

The indices of the minimum number of rows needed: []


# Linear programming approach

Using an optimization library like scipy.optimize.linprog, we can solve the problem using linear programming. The objective function is to minimize the sum of the rows to be selected. The constraints are that the sum of the selected rows should be greater than or equal to 1 for each column. Here is the code:
    

In [2]:
import numpy as np
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD


$$
Min: \sum_{i \in \{1, \ldots, m\}}^{\text{msg}} m_{j}
$$
Subject to:
$$
\sum_{i \in \{1, \ldots, m\}}^{\text{msg}} \{m_{j}| X_{ij} = 1\} >= 1\quad \forall j \in \{1,...n\}
$$

In [3]:
def  get_weakness_number(X):
    n_msg, n_tag = X.shape
    A = np.transpose(X)
    B = np.ones(n_tag)
    C = np.ones(n_msg)

    prob = LpProblem("Binary LP Problem", LpMinimize)

    # Define the variables
    x = [LpVariable(f'x{i}', cat='Binary') for i in range(len(C))]

    # Define the objective function
    prob += lpSum(C[i] * x[i] for i in range(len(C)))

    # Define the constraints
    for i in range(len(A)):
        prob += lpSum(A[i][j] * x[j] for j in range(len(C))) >= B[i]

    # Solve the problem
    prob.solve(PULP_CBC_CMD())

    # Print the results
    print("Status:", prob.status)
    print("Objective value:", prob.objective.value())

    result = []
    for i in range(len(C)):
        #print(f"x{i+1}:", x[i].value())
        if x[i].value() == 1:
            result.append(i+1)
    return result



In [4]:
K1 = np.array(
    [[1,0,1,0,0],
    [1,0,0,1,0],
    [1,0,0,0,1],
    [0,1,1,0,0],
    [0,1,0,1,0],
    [0,1,0,0,1]])
print(get_weakness_number(K1))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Python/3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/1r/ph1yw3353jb398m67yh849kc0000gn/T/53e80d847ecb4d7eb4818d89fb0ac0e9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/1r/ph1yw3353jb398m67yh849kc0000gn/T/53e80d847ecb4d7eb4818d89fb0ac0e9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 41 RHS
At line 47 BOUNDS
At line 54 ENDATA
Problem MODEL has 5 rows, 6 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3 - 0.00 seconds
Cgl0004I processed model has 5 rows, 6 columns (6 integer (6 of which binary)) and 12 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 3
Cbc0038I Before mini branch and bound, 6 integers at bound fi

/Library/Python/3.9/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [5]:
K2 = np.array([[ 1,  0,  0,  0,  0,  0,  1,  0,  0],# m1
              [ 1,  0,  0,  0,  0,  0,  0,  1,  0], # m2
              [ 1,  0,  0,  0,  0,  0,  0,  0,  1], # m3
              [ 0,  1,  0,  0,  0,  0,  1,  0,  0], # m4
              [ 0,  1,  0,  0,  0,  0,  0,  1,  0], # m5
              [ 0,  1,  0,  0,  0,  0,  0,  0,  1], # m6
              [ 0,  0,  1,  0,  0,  0,  1,  0,  0], # m7
              [ 0,  0,  1,  0,  0,  0,  0,  1,  0], # m8
              [ 0,  0,  1,  0,  0,  0,  0,  0,  1], # m9
              [ 0,  0,  0,  1,  0,  0,  1,  0,  0], # m10
              [ 0,  0,  0,  1,  0,  0,  0,  1,  0], # m11
              [ 0,  0,  0,  1,  0,  0,  0,  0,  1], # m12
              [ 0,  0,  0,  0,  1,  0,  1,  0,  0], # m13
              [ 0,  0,  0,  0,  1,  0,  0,  1,  0], # m14
              [ 0,  0,  0,  0,  1,  0,  0,  0,  1], # m15
              [ 0,  0,  0,  0,  0,  1,  1,  0,  0], # m16
              [ 0,  0,  0,  0,  0,  1,  0,  1,  0], # m17
              [ 0,  0,  0,  0,  0,  1,  0,  0,  1]])# m18
get_weakness_number(K2)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Python/3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/1r/ph1yw3353jb398m67yh849kc0000gn/T/a5498b8c2a2d4fdc8f05479cf8bdc03d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/1r/ph1yw3353jb398m67yh849kc0000gn/T/a5498b8c2a2d4fdc8f05479cf8bdc03d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 105 RHS
At line 115 BOUNDS
At line 134 ENDATA
Problem MODEL has 9 rows, 18 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 6 - 0.00 seconds
Cgl0004I processed model has 9 rows, 18 columns (18 integer (18 of which binary)) and 36 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 6
Cbc0038I Before mini branch and bound, 18 integers at 

[2, 6, 9, 12, 15, 16]

In [6]:
            #  t1  t2  t3  t4  t5  t6  t7  t8  t9
K3 = np.array([[ 1,  0,  0,  0,  0,  0,  0,  0],# m1
              [ 1,  1,  0,  0,  0,  0,  0,  0], # m2
              [ 1,  1,  1,  0,  0,  0,  0,  0], # m3
              [ 0,  1,  1,  1,  0,  0,  0,  0], # m4
              [ 0,  0,  1,  1,  1,  0,  0,  0], # m5
              [ 0,  0,  0,  1,  1,  1,  0,  0], # m6
              [ 0,  0,  0,  0,  1,  1,  1,  0], # m7
              [ 0,  0,  0,  0,  0,  1,  1,  1], # m8
              [ 0,  0,  0,  0,  0,  0,  1,  1], # m9
              [ 0,  0,  0,  0,  0,  0,  0,  1]])# m10
get_weakness_number(K3)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Python/3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/1r/ph1yw3353jb398m67yh849kc0000gn/T/75d91852918c49d4ae48173ddd4368ad-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/1r/ph1yw3353jb398m67yh849kc0000gn/T/75d91852918c49d4ae48173ddd4368ad-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 68 RHS
At line 77 BOUNDS
At line 88 ENDATA
Problem MODEL has 8 rows, 10 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3 - 0.00 seconds
Cgl0004I processed model has 8 rows, 10 columns (10 integer (10 of which binary)) and 24 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 3
Cbc0038I Before mini branch and bound, 10 integers at bou

[2, 5, 8]

In [7]:
X = np.array([[0,1,1,0],
[0,0,0,0],
[0,0,0,1],
[1,0,1,1],
[1,1,1,0],
[1,0,0,0],
[0,0,0,1],
[0,1,0,0]])

X = np.array([[1, 1, 0, 1, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1],
       [1, 0, 1, 1, 1],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1],
       [1, 0, 0, 0, 0],
       [1, 1, 0, 1, 1],
       [0, 1, 0, 0, 0]])

get_weakness_number(X)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Python/3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/1r/ph1yw3353jb398m67yh849kc0000gn/T/515a02f5b576411aadd6dcecebb4fad4-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/1r/ph1yw3353jb398m67yh849kc0000gn/T/515a02f5b576411aadd6dcecebb4fad4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 64 RHS
At line 70 BOUNDS
At line 81 ENDATA
Problem MODEL has 5 rows, 10 columns and 23 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2 - 0.00 seconds
Cgl0004I processed model has 5 rows, 8 columns (8 integer (6 of which binary)) and 18 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 2
Cbc0038I Cleaned solution of 2
Cbc0038I Before mini branch a

[3, 4]